In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
from tensorboardX import SummaryWriter
import pickle,os,shutil
torch.manual_seed(1)

# What is a Batch Normalization
- batch normalization은 기본적으로 gradient vanishing/gradient exploding이 일어나지 않도록 하는 아이디어 중 하나임.
- 학습이 불안정한 이유는 'internal Covariance shift' 때문이라는 의견이 주장됨.
- Covariance shift는 network의 각 층이나 activation마다 input의 distribution이 달라지는 현상을 의미함. (batch마다 분포가 달라진다?)
- 이 현상을 막기 위해 간단하게 각 층의 input의 distribution의 평균을 0, 표준편차를 1인 input으로 normalize시키는 방법 -> 'whitening'
- 'whitening'을 하기 위해서는 covariance matrix의 계산과 inverse계산이 필요하기 때문에 연산량이 많음.
- whitening의 단점을 보완하고 internal covariance shift를 줄이기 위해 다음과 같이 접근함.  
  
    - 각각의 feature들이 이미 uncorrelated라고 가정하고, feature각각에 대해서만 scalar형태로 mean, variance를 구하고 각각 normalize
    - 단순히 mean과 variance를 0, 1로 고정시키는 것은 오히려 activation의 nonlinearity를 없앨 수 있어서 normalize된 값들에 scale factor(gamma)와 shift(beta)를 더해주고 이 변수들을 back-prop과정에서 같이 train시킴.
    - training data전체에 대해 mean과 variance를 구하는 것이 아니라, mini-batch단위로 접근하여 계산함.
    
<img src='./images/batchnorm_alg.png' width =400 align='left'>


- 실제로 BatchNormalization은 네트워크에 적용시킬 때는, activation function을 하기 전에 적용한다.  

<img src='./images/batchnorm_feed.png' width =400 align='left'>


- training할 때는 mini-batch에서 평균과 표준편차를 이용하지만 test data를 사용하여 inference할 때는 training할 때 현자까지 본 input들의 이동평균 및 unbiased variance estimate의 이동평균을 계산해 저장해 놓은 뒤 이 값으로 normalize를 함. 


## BatchNormalization의 장점
- 기존 Deep Neural Network에서 leraning rate를 크게 잡을 경우 gradient가 explode/vanish 등의 문제가 발생함. 이는 parameter의 scale때문인데, batchnorm을 사용할 경우 back-prop할 때 scale에 영항을 받지 않게 된다. 따라서, learning rate를 크게 잡을 수 있게 되고 이는 빠른 학습을 가능하게 함.
- Batch Normalization은 Regularization효과가 있음. (drop out효과와 같으며 weight regularization term등을 제외할 수 있음.)

# 0. 텐서보드

In [6]:
port= '6006'
try:
    shutil.rmtree('runs/')
except:
    pass

# 1. 데이터

In [7]:
BATCH_SIZE= 64

train_dataset = vdatasets.MNIST(root='../data/MNIST/',
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True,
                                           num_workers=2,
                                           drop_last=True) # 이동평균이 튀는걸 방지

test_dataset = vdatasets.MNIST(root='../data/MNIST/',
                               train=False, 
                               transform=transforms.ToTensor(),
                               download=True)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True,
                                           num_workers=2)

# 2. model

In [20]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NN, self).__init__()
        self.linear1= nn.Linear(input_size, hidden_size)
        self.linear2= nn.Linear(hidden_size, hidden_size)
        self.linear3= nn.Linear(hidden_size, output_size)
        
        # In : {배치사이즈, 차원수} => Out : (배치사이즈, 차원수)
        self.bn1= nn.BatchNorm1d(hidden_size)
        self.bn2= nn.BatchNorm1d(hidden_size)
    def forward(self, inputs):
        outputs= self.bn1(self.linear1(inputs))
        outputs= F.relu(outputs)
        outputs= self.bn2(self.linear2(outputs))
        outputs= F.relu(outputs)
        return self.linear3(outputs)

In [21]:
EPOCH=5
LR=0.1

INPUT_SIZE= 784
HIDDEN_SIZE= 512
OUTPUT_SIZE=10

model=NN(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
loss_function= nn.CrossEntropyLoss()
optimizer= optim.SGD(model.parameters(), lr=LR)

# 3. 학습

In [22]:
writer= SummaryWriter(comment="-batch-norm")

In [23]:
%%time
for epoch in range(EPOCH):
    losses=[]
    for i, (inputs, targets) in enumerate(train_loader):
        model.zero_grad()
        pred= model(inputs.view(len(inputs),-1))
        loss= loss_function(pred, targets)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data.item())
        if i % 500==0:
            avg_loss= np.mean(losses)
            print("[%d/%d] [%03d/%d] mean_loss : %.3f" % (epoch,EPOCH,i,len(train_loader),avg_loss))
            writer.add_scalars('data/batch_norm/',{'batch_norm': avg_loss}, (i+1)+(epoch*len(train_loader)))

[0/5] [000/937] mean_loss : 2.276
[0/5] [500/937] mean_loss : 0.264
[1/5] [000/937] mean_loss : 0.099
[1/5] [500/937] mean_loss : 0.084
[2/5] [000/937] mean_loss : 0.081
[2/5] [500/937] mean_loss : 0.056
[3/5] [000/937] mean_loss : 0.015
[3/5] [500/937] mean_loss : 0.038
[4/5] [000/937] mean_loss : 0.023
[4/5] [500/937] mean_loss : 0.030
CPU times: user 1min 6s, sys: 4.05 s, total: 1min 10s
Wall time: 46.6 s


In [24]:
port

'6006'

In [ ]:
!tensorboard --logdir runs --port 6006

/Users/a408109/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 1.9.0 at http://408109-HC-D16A57:6006 (Press CTRL+C to quit)
